In [2]:
import pandas as pd
from scipy.stats import normaltest

In [17]:
# чтение CSV-файла и создание DataFrame
df = pd.read_csv("train.csv", index_col=0)
df

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,,
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,0
3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0
4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0
...,...,...,...,...,...,...,...,...,...,...,...
15299,Female,22.0,0,0,No,Govt_job,Urban,72.63,19.5,never smoked,0
15300,Female,46.0,1,0,Yes,Private,Urban,101.19,32.1,never smoked,0
15301,Female,75.0,0,0,Yes,Self-employed,Urban,87.69,26.2,never smoked,0


In [29]:
# Преобразуем гендер, статус замужества в числовое значение
df2 = df.replace({'gender': {'Male': 0, 'Other': 0.5, 'Female': 1}})
df2 = df2.replace({'ever_married': {'Yes': 1, 'No': 0}})
dummies = pd.get_dummies(df['work_type'])
df2 = df2.drop('work_type', axis=1)
df2 = pd.concat([df2, dummies], axis=1)
df2 = df2.replace({'Residence_type': {'Urban': 1, 'Rural': 0}})
df2 = df2.replace({'smoking_status': {'never smoked': 0, 'Unknown': 0.33, 'formerly smoked': 0.66, 'smokes': 1}})
df2

,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,Govt_job,Never_worked,Private,Self-employed,children
id,,,,,,,,,,,,,,,
0,0.0,28.0,0,0,1,1,79.53,31.1,0.00,0,0,0,1,0,0
1,0.0,33.0,0,0,1,0,78.44,23.9,0.66,0,0,0,1,0,0
2,1.0,42.0,0,0,1,0,103.00,40.3,0.33,0,0,0,1,0,0
3,0.0,56.0,0,0,1,1,64.87,28.8,0.00,0,0,0,1,0,0
4,1.0,24.0,0,0,0,0,73.36,28.8,0.00,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15299,1.0,22.0,0,0,0,1,72.63,19.5,0.00,0,1,0,0,0,0
15300,1.0,46.0,1,0,1,1,101.19,32.1,0.00,0,0,0,1,0,0
15301,1.0,75.0,0,0,1,1,87.69,26.2,0.00,0,0,0,0,1,0


In [68]:
cpearson_corr = df2.corr(method='pearson')['stroke'] # корреляция по методу Пирсона
kendall_corr = df2.corr(method='kendall')['stroke'] # корреляция по методу Кендалла
spearman_corr = df2.corr(method='spearman')['stroke'] # корреляция по методу Спирмена
corr_result = pd.concat([cpearson_corr, kendall_corr, spearman_corr], keys=['pearson', 'kendall', 'spearman'], axis=1)
corr_result = corr_result.sort_values(by='kendall', ascending=False)[1:]
corr_result

,pearson,kendall,spearman
age,0.261320,0.211940,0.257773
hypertension,0.145895,0.145895,0.145895
ever_married,0.114010,0.114010,0.114010
heart_disease,0.111518,0.111518,0.111518
Self-employed,0.076933,0.076933,0.076933
bmi,0.065393,0.061522,0.075166
avg_glucose_level,0.149273,0.040546,0.049645
smoking_status,0.032136,0.024556,0.026557
Govt_job,0.006227,0.006227,0.006227
Private,0.000873,0.000873,0.000873


In [40]:
# Проверка на наличие нормального распределения
columns_new = []
values_new = []
for column in df2.columns :
    p_value_factor = normaltest(df2[column]).pvalue
    columns_new.append(column)
    if p_value_factor > 0.05:
        values_new.append("+")
    else:
        values_new.append("-")
result = pd.DataFrame({'Фича': columns_new, 'имеет нормальное распределение': values_new})
result
# Проверка показала, что все поля не имеют нормального распределения,
# следовательно мы можем использвать либо корреляцию по методу Кендалла, либо по методу Спирмена.
# При этом, на таблице выше видно, что методы Кендалла и Спирмена имеют одинаковую последовательност


,Фича,имеет нормальное распределение
0,gender,-
1,age,-
2,hypertension,-
3,heart_disease,-
4,ever_married,-
5,Residence_type,-
6,avg_glucose_level,-
7,bmi,-
8,smoking_status,-
9,stroke,-


In [71]:
features = kendall_corr[kendall_corr > 0.01]
features.sort_values(ascending=False)[1:]

age                  0.211940
hypertension         0.145895
ever_married         0.114010
heart_disease        0.111518
Self-employed        0.076933
bmi                  0.061522
avg_glucose_level    0.040546
smoking_status       0.024556
Name: stroke, dtype: float64